In [ ]:
import numpy as np
import pandas as pd
import glob
import mne
import utils

Data pre-processing

In [ ]:
#create dataframe from CSV files
file_path= 'data/eeg_data_A/A*.csv'
eeg_dataframe_A = utils.csv_to_df(file_path)
eeg_dataframe_A.head()
eeg_dataframe_A = utils.format_df(eeg_dataframe_A)
eeg_dataframe_A.head()

In [ ]:
# Band-pass filtering
# High pass filter
